Colab notebook - semantic search

In [ ]:
!python -m pip install -q apache_beam pysolr scikit-learn tensorflow annoy numpy pandas==2.2.0 pyarrow==14.0.0 tensorflow_transform
# Print versions
!python -m pip show tensorflow
!python -m pip show annoy
!python -m pip show tensorflow_transform
!python -m pip show pandas
!python -m pip show pyarrow

In [ ]:
import os
import time
import sys
import pathlib
import pickle
from collections import namedtuple
from datetime import datetime

import numpy as np
import apache_beam as beam
import annoy
from sklearn.random_projection import GaussianRandomProjection
import tensorflow as tf

import tensorflow_hub as hub

In [ ]:
#!python -m pip install -q keras
# !python -m pip install tensorflow_transform
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam

## Download Solr9

In [ ]:
%%bash

wget -q https://pulmirror.princeton.edu/mirror/solr/dist/lucene/solr/9.0.0/solr-9.0.0.tgz
wget -q https://pulmirror.princeton.edu/mirror/solr/dist/lucene/solr/9.0.0/solr-9.0.0.tgz.sha512
tar xzf solr-9.0.0.tgz solr-9.0.0/bin/install_solr_service.sh --strip-components=2
#sudo chown -R daemon:daemon solr-9.0.0
shasum -a 512 -c solr-9.0.0.tgz.sha512
#sudo bash ./install_solr_service.sh solr-9.0.0.tgz
sudo bash ./install_solr_service.sh solr-9.0.0.tgz -i /opt -d /var/solr -u solr -s solr -p 8983
#echo $PWD
#ls -al $CWD


In [ ]:
time.sleep(20)
sudo service solr status

In [ ]:
%%bash

echo ls -l /root

In [ ]:
%%bash --bg
sudo bash ./install_solr_service.sh solr-9.0.0.tgz

#sudo bash ./install_solr_service.sh solr-9.0.0.tgz -i /opt -d /var/solr -u solr -s solr -p 8983

# sudo -H -u daemon solr-9.0.0/bin/install_solr_service.sh #-p 8984
# daemon "java -jar solr-9.0.0/bin/solr"


In [ ]:
%%bash

echo $HOME
java -version

In [ ]:
time.sleep(20)
sudo service solr status

In [ ]:
%%bash

ps -ef | grep solr

## Download ElasticSearch

In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
time.sleep(20)

In [ ]:
%%bash

ps -ef | grep elasticsearch

In [ ]:
%%bash

curl -sX GET "localhost:9200/"